In [ ]:
from datetime import date
from random import randint
from ortools.linear_solver import pywraplp
from terms.models import Term

worker_list = [
    ('Adam', 'Sy'),
    ('Ben', 'Findeisen'),
    ('Bill', 'Yeh'),
    ('Carlos', 'Marin'),
    ('Carson', 'Maher'),
    ('Clarence', 'So'),
    ('Daniel', 'Tai'),
    ('David', 'Welk'),
    ('David', 'Lee'),
    ('David', 'Ye'),
    ('Edgardo', 'Mendoza'),
    ('Emmanuel', 'Augustine'),
    ('Enoch', 'Huang'),
    ('Isaac', 'Tsou'),
    ('Jason', 'Yeung'),
    ('Jasper', 'Kuhn'),
    ('Jesse', 'Avila'),
    ('Johnathan', 'Lin'),
    ('Jonathan', 'Karr'),
    ('Josh', 'Carbunck'),
    ('Josh', 'Cherng'),
    ('Julius', 'Chang'),
    ('Jun', 'Heo'),
    ('Justin', 'Washington'),
    ('Kevin', 'Sung'),
    ('Kyle', 'Yen'),
    ('Luke', 'Lu'),
    ('Mark', 'Fan'),
    ('Michael', 'Min'),
    ('Miles', 'Mistler'),
    ('Nathan', 'Bodrug'),
    ('Paul', 'Wen'),
    ('Preston', 'Wang'),
    ('Rafael', 'Diaz'),
    ('Ray', 'Ding'),
    ('Raymond', 'Chang'),
    ('Rui', 'Jiang'),
    ('Ryan', 'Holt'),
    ('Ryan', 'Li'),
    ('Samuel', 'Swei'),
    ('Sean', 'David'),
    ('Sean', 'Min'),
    ('Stephen', 'Kwan'),
    ('Yi', 'Sun'),
    ('Zion', 'Park'),
]
workers = []
for w in worker_list:
    t = Trainee.objects.get(firstname=w[0], lastname=w[1])
    workers.append(t)
week = Term.current_term().term_week_of_date(date.today())
classes = list(Class.objects.all()\
     .exclude(av_code='')\
     .exclude(class_type='')\
     .exclude(class_type=None)\
     .filter(schedules__weeks__contains=week)\
     .distinct()\
)
history = [
    [randint(0, 3) for i in range(len(classes))] for j in range(len(workers))
]

print(workers)
print(classes)
print(history)

def assign(workers, classes, history):
    x = {}
    num_workers = len(workers)
    num_classes = len(classes)
    solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    def worker_can_do_class(worker, cls):
        if cls.class_type == 'MAIN':
            return True
        elif cls.class_type == '1YR' and worker.current_term <= 2:
            return True
        elif cls.class_type == '2YR' and worker.current_term > 2:
            return True
        else:
            return False
    
    for i in range(num_workers):
        worker = workers[i]
        for j in range(num_classes):
            cls = classes[j]
            if worker_can_do_class(worker, cls):
                x[i, j] = solver.BoolVar('x[ %i, %i]' % (i, j))
            else:
                x[i, j] = solver.IntVar(0, 0, 'x[ %i, %i]' % (i, j))

    costs = []
    for i in range(num_workers):
        workload = sum(history[i]) + sum([x[i, j] for j in range(num_classes)])
        cost = []
        for j in range(num_classes):
            cost.append(workload + 2 * (history[i][j] + x[i, j]))
        costs.append(cost)

    solver.Minimize(solver.Sum([costs[i][j] for i in range(num_workers)
                                 for j in range(num_classes)]))
    for j in range(num_classes):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) <= 2)

assign(workers, classes, history)

In [3]:
a = set()
a.union(set([1]))
a

set()